<a href="https://colab.research.google.com/github/RusAl84/text_mining/blob/master/10_04_2022_%D0%9C%D0%B0%D1%81%D1%82%D0%B5%D1%80_%D0%BA%D0%BB%D0%B0%D1%81%D1%81_%D0%92%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D1%83_%D0%B5%D1%81%D1%82%D0%B5%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE_%D1%8F%D0%B7%D1%8B%D0%BA%D0%B0_%D0%BD%D0%B0_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Подход TF-IDF

У подхода bag-of-words есть существенный недостаток. Если слово встречается 5 раз в конкретном документе, но и в других документах тоже встречается часто, то его наличие в документе не особо-то о чём-то говорит. Если же слово 5 раз встречается в конкретном документе, но в других документах встречается редко, то его наличие (да ещё и многократное) позволяет хорошо отличать этот документ от других. Однако с точки зрения bag-of-words различий не будет: в обеих ячейках будет просто число 5.

Отчасти это решается исключением стоп-слов (и слишком часто встречающихся слов), но лишь отчасти. Другой идеей является отмасштабировать получившуюся таблицу с учётом "редкости" слова в наборе документов (т.е. с учётом информативности слова).

tfidf=tf∗idf

idf=log((N+1)/(Nw+1))+1

Здесь tf это частота слова в тексте (то же самое, что в bag of words), N - общее число документов, Nw - число документов, содержащих данное слово.

То есть для каждого слова считается отношение общего количества документов к количеству документов, содержащих данное слово (для частых слов оно будет ближе к 1, для редких слов оно будет стремиться к числу, равному количеству документов), и на логарифм от этого числа умножается исходное значение bag-of-words (к числителю и знаменателю прибавляется единичка, чтобы не делить на 0, и к логарифму тоже прибавляется единичка, но это уже технические детали). После этого в sklearn ещё проводится L2-нормализация каждой строки.

В sklearn есть класс для поддержки TF-IDF: TfidfVectorizer, рассмотрим его.

In [1]:
texts = "25 \u0438\u043D\u0442\u0435\u0440\u0435\u0441\u043D\u044B\u0445 \u0444\u0430\u043A\u0442\u043E\u0432 \u043E \u0435\u0436\u0430\u0445 \u0418\u043D\u0442\u0435\u0440\u0435\u0441\u043D\u044B\u0435 \u0444\u0430\u043A\u0442\u044B \u043E \u0435\u0436\u0430\u0445  \u041F\u0440\u0438 \u0441\u043B\u043E\u0432\u0435 \xAB\u0451\u0436\xBB \u043C\u043D\u043E\u0433\u0438\u043C \u0438\u0437 \u043D\u0430\u0441 \u0441\u0440\u0430\u0437\u0443 \u043F\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043B\u044F\u0435\u0442\u0441\u044F \u043A\u0430\u0440\u0442\u0438\u043D\u043A\u0430 \u0438\u0437 \u0434\u0435\u0442\u0441\u043A\u043E\u0439 \u043A\u043D\u0438\u0436\u043A\u0438, \u043D\u0430 \u043A\u043E\u0442\u043E\u0440\u043E\u0439 \u0432\u0435\u0441\u0451\u043B\u043E\u0435 \u0435\u0436\u0438\u043D\u043E\u0435 \u0441\u0435\u043C\u0435\u0439\u0441\u0442\u0432\u043E \u0442\u0430\u0449\u0438\u0442 \u0432 \u043D\u043E\u0440\u043A\u0443 \u043D\u0430\u043A\u043E\u043B\u043E\u0442\u044B\u0435 \u043D\u0430 \u0438\u0433\u043E\u043B\u043A\u0438 \u0433\u0440\u0438\u0431\u043E\u0447\u043A\u0438. \u041D\u0430 \u0441\u0430\u043C\u043E\u043C \u0436\u0435 \u0434\u0435\u043B\u0435 \u0435\u0436\u0438 \u2014 \u044D\u0442\u043E \u0445\u0438\u0442\u0440\u044B\u0435 \u0438 \u0443\u043C\u043D\u044B\u0435 \u0445\u0438\u0449\u043D\u0438\u043A\u0438, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u043E\u0445\u043E\u0442\u043D\u043E \u043F\u0438\u0442\u0430\u044E\u0442\u0441\u044F \u043D\u0435 \u0442\u043E\u043B\u044C\u043A\u043E \u0433\u0440\u0438\u0431\u0430\u043C\u0438 \u0438 \u044F\u0433\u043E\u0434\u0430\u043C\u0438, \u043D\u043E \u0438 \u0432\u043E\u043E\u0431\u0449\u0435 \u0432\u0441\u0435\u043C, \u0447\u0442\u043E \u0441\u0443\u043C\u0435\u044E\u0442 \u043D\u0430\u0439\u0442\u0438. \u0418\u043B\u0438 \u043A\u043E\u0433\u043E \u0441\u0443\u043C\u0435\u044E\u0442 \u043F\u043E\u0439\u043C\u0430\u0442\u044C.  \u0418\u043D\u0442\u0435\u0440\u0435\u0441\u043D\u044B\u0435 \u0444\u0430\u043A\u0442\u044B \u043E \u0435\u0436\u0430\u0445 \u041D\u0430 \u0432\u0435\u0440\u0445\u043D\u0435\u0439 \u0447\u0435\u043B\u044E\u0441\u0442\u0438 \u0435\u0436\u0435\u0439 \u0440\u0430\u0441\u043F\u043E\u043B\u043E\u0436\u0435\u043D\u044B 20 \u043C\u0430\u043B\u0435\u043D\u044C\u043A\u0438\u0445 \u0437\u0443\u0431\u043E\u0432, \u043D\u0430 \u043D\u0438\u0436\u043D\u0435\u0439 \u2013 16. \u041F\u0440\u0438 \u0434\u043B\u0438\u043D\u0435 \u0442\u0435\u043B\u0430 \u0432 20-30 \u0441\u0430\u043D\u0442\u0438\u043C\u0435\u0442\u0440\u043E\u0432 \u0443 \u0435\u0436\u0435\u0439 \u0434\u043E\u0432\u043E\u043B\u044C\u043D\u043E \u043C\u0430\u043B\u0435\u043D\u044C\u043A\u0438\u0435 \u0443\u0448\u0438 \u2013 \u043C\u0435\u043D\u044C\u0448\u0435 3,5 \u0441\u0430\u043D\u0442\u0438\u043C\u0435\u0442\u0440\u043E\u0432. \u0423\u0448\u0438 \u0431\u043E\u043B\u044C\u0448\u0435\u0433\u043E \u0440\u0430\u0437\u043C\u0435\u0440\u0430 \u0432\u0441\u0442\u0440\u0435\u0447\u0430\u044E\u0442\u0441\u044F \u0442\u043E\u043B\u044C\u043A\u043E \u0443 \u0435\u0436\u0435\u0439, \u0436\u0438\u0432\u0443\u0449\u0438\u0445 \u043D\u0430 \u041A\u0438\u043F\u0440\u0435 (\u0441\u043C. \u0438\u043D\u0442\u0435\u0440\u0435\u0441\u043D\u044B\u0435 \u0444\u0430\u043A\u0442\u044B \u043E \u041A\u0438\u043F\u0440\u0435). \u0412\u043D\u0443\u0442\u0440\u0438 \u0438\u0433\u043B\u044B \u0435\u0436\u0435\u0439 \u043F\u043E\u043B\u044B\u0435 \u2013 \u043E\u043D\u0438 \u043D\u0430\u043F\u043E\u043B\u043D\u0435\u043D\u044B \u0442\u043E\u043B\u044C\u043A\u043E \u0432\u043E\u0437\u0434\u0443\u0445\u043E\u043C. \u0422\u0435\u043B\u0430 \u043C\u043E\u043B\u043E\u0434\u044B\u0445 \u0435\u0436\u0435\u0439 \u0437\u0430\u0449\u0438\u0449\u0430\u044E\u0442 \u043E\u0442 \u0445\u0438\u0449\u043D\u0438\u043A\u043E\u0432 \u043E\u043A\u043E\u043B\u043E 3 000 \u0438\u0433\u043E\u043B\u043E\u043A, \u0432\u0437\u0440\u043E\u0441\u043B\u044B\u0445 \u2013 5-6 \u0442\u044B\u0441\u044F\u0447 \u0438\u0433\u043E\u043B\u043E\u043A. \u0412 \u043F\u043E\u0441\u043B\u0435\u0434\u043D\u0438\u0435 \u0433\u043E\u0434\u044B \u0432\u0441\u0435 \u0431\u043E\u043B\u044C\u0448\u0435 \u0435\u0436\u0435\u0439 \u043F\u0435\u0440\u0435\u0431\u0438\u0440\u0430\u0435\u0442\u0441\u044F \u0438\u0437 \u043B\u0435\u0441\u043E\u0432 \u0432 \u0433\u043E\u0440\u043E\u0434\u0430. \u0423\u0447\u0435\u043D\u044B\u0435 \u0441\u0432\u044F\u0437\u044B\u0432\u0430\u044E\u0442 \u044D\u0442\u043E \u0441 \u0442\u0435\u043C, \u0447\u0442\u043E \u043D\u0430 \u0433\u043E\u0440\u043E\u0434\u0441\u043A\u0438\u0445 \u0443\u043B\u0438\u0446\u0430\u0445 \u043D\u0435 \u0432\u043E\u0434\u044F\u0442\u0441\u044F \u043E\u0441\u043D\u043E\u0432\u043D\u044B\u0435 \u0432\u0440\u0430\u0433\u0438 \u0435\u0436\u0435\u0439 \u2013 \u0431\u0430\u0440\u0441\u0443\u043A\u0438 \u0438 \u0441\u043E\u0432\u044B (\u0441\u043C. \u0438\u043D\u0442\u0435\u0440\u0435\u0441\u043D\u044B\u0435 \u0444\u0430\u043A\u0442\u044B \u043E \u0441\u043E\u0432\u0430\u0445). \u0415\u0436\u0438 \u2013 \u043D\u043E\u0447\u043D\u044B\u0435 \u0436\u0438\u0432\u043E\u0442\u043D\u044B\u0435, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u043D\u0435 \u043B\u044E\u0431\u044F\u0442 \u043E\u0442\u0445\u043E\u0434\u0438\u0442\u044C \u0434\u0430\u043B\u0435\u043A\u043E \u043E\u0442 \u0441\u0432\u043E\u0435\u0433\u043E \u0436\u0438\u043B\u0438\u0449\u0430. \u0414\u0435\u043D\u044C \u0435\u0436\u0438 \u043E\u0431\u044B\u0447\u043D\u043E \u043F\u0440\u043E\u0432\u043E\u0434\u044F\u0442 \u0432 \u0433\u043D\u0435\u0437\u0434\u0435, \u0438\u043C\u0435\u044E\u0449\u0435\u043C \u0432 \u0434\u0438\u0430\u043C\u0435\u0442\u0440\u0435 15-20 \u0441\u0430\u043D\u0442\u0438\u043C\u0435\u0442\u0440\u043E\u0432. \u0415\u0436\u0438 \u043E\u0447\u0438\u0449\u0430\u044E\u0442 \u0438 \u0440\u0430\u0441\u043F\u0440\u0430\u0432\u043B\u044F\u044E\u0442 \u0441\u0432\u043E\u0438 \u0438\u0433\u043B\u044B \u0434\u043B\u0438\u043D\u043D\u044B\u043C\u0438 \u043F\u0430\u043B\u044C\u0446\u0430\u043C\u0438 \u043D\u0430 \u043D\u043E\u0433\u0430\u0445. \u0415\u0436\u0438 \u0440\u0435\u0432\u043D\u043E\u0441\u0442\u043D\u043E \u043E\u0431\u0435\u0440\u0435\u0433\u0430\u044E\u0442 \u0441\u0432\u043E\u044E \u0442\u0435\u0440\u0440\u0438\u0442\u043E\u0440\u0438\u044E \u043E\u0442 \u0441\u043E\u0440\u043E\u0434\u0438\u0447\u0435\u0439. \u041F\u043B\u043E\u0449\u0430\u0434\u044C \u0443\u0447\u0430\u0441\u0442\u043A\u0430 \u0435\u0436\u0435\u0439-\u0441\u0430\u043C\u0446\u043E\u0432 \u0441\u043E\u0441\u0442\u0430\u0432\u043B\u044F\u0435\u0442 \u043E\u0442 7 \u0434\u043E 39 \u0433\u0430, \u0443 \u0441\u0430\u043C\u043E\u043A \u2013 \u043E\u0442 6 \u0434\u043E 10 \u0433\u0430. \u0415\u0436\u0438 \u043F\u043E\u0441\u0442\u043E\u044F\u043D\u043D\u043E, \u0445\u043E\u0442\u044C \u0438 \u043A\u0440\u0430\u0439\u043D\u0435 \u043C\u0435\u0434\u043B\u0435\u043D\u043D\u043E, \u043B\u0438\u043D\u044F\u044E\u0442. \u041D\u0430 \u043E\u0442\u0440\u0430\u0449\u0438\u0432\u0430\u043D\u0438\u0435 \u043D\u043E\u0432\u043E\u0439 \u0438\u0433\u043B\u044B \u0443\u0445\u043E\u0434\u0438\u0442 \u043E\u0442 \u0433\u043E\u0434\u0430 \u0434\u043E \u043F\u043E\u043B\u0443\u0442\u043E\u0440\u0430 \u043B\u0435\u0442. \u0412 \u0441\u0440\u0435\u0434\u043D\u0435\u043C \u0437\u0430 \u0433\u043E\u0434 \u0451\u0436 \u0441\u043C\u0435\u043D\u044F\u0435\u0442 \u043A\u0430\u0436\u0434\u0443\u044E \u0442\u0440\u0435\u0442\u044C\u044E \u0438\u0433\u043B\u0443 \u043D\u0430 \u0441\u0432\u043E\u0435\u043C \u0442\u0435\u043B\u0435. \u0412 \u043F\u0440\u0438\u0440\u043E\u0434\u043D\u044B\u0445 \u0443\u0441\u043B\u043E\u0432\u0438\u044F\u0445 \u0435\u0436\u0438 \u0436\u0438\u0432\u0443\u0442 \u043C\u0430\u043A\u0441\u0438\u043C\u0443\u043C 5 \u043B\u0435\u0442, \u0430 \u0432 \u0434\u043E\u043C\u0430\u0448\u043D\u0438\u0445 \u0443\u0441\u043B\u043E\u0432\u0438\u044F\u0445 \u0438\u043B\u0438 \u0437\u043E\u043E\u043F\u0430\u0440\u043A\u0435 \u2014 \u0434\u043E 10 \u043B\u0435\u0442. \u0415\u0436\u0438 \u2013 \u0434\u043E\u0432\u043E\u043B\u044C\u043D\u043E \u043F\u0440\u043E\u0432\u043E\u0440\u043D\u044B\u0435 \u0437\u0432\u0435\u0440\u0438. \u041E\u043D\u0438 \u043C\u043E\u0433\u0443\u0442 \u043F\u0440\u043E\u0431\u0435\u0433\u0430\u0442\u044C \u0434\u043E 3 \u043C\u0435\u0442\u0440\u043E\u0432 \u0432 \u0441\u0435\u043A\u0443\u043D\u0434\u0443, \u044F\u0432\u043B\u044F\u044E\u0442\u0441\u044F \u0445\u043E\u0440\u043E\u0448\u0438\u043C\u0438 \u043F\u043B\u043E\u0432\u0446\u0430\u043C\u0438 \u0438 \u043F\u0440\u044B\u0433\u0443\u043D\u0430\u043C\u0438. \u0422\u0430\u043A \u043A\u0430\u043A \u0435\u0436\u0438 \u0430\u043A\u0442\u0438\u0432\u043D\u044B \u043D\u043E\u0447\u044C\u044E, \u043F\u043B\u043E\u0445\u043E\u0435 \u0437\u0440\u0435\u043D\u0438\u0435 \u0443 \u043D\u0438\u0445 \u043A\u043E\u043C\u043F\u0435\u043D\u0441\u0438\u0440\u0443\u044E\u0442 \u043F\u0440\u0435\u043A\u0440\u0430\u0441\u043D\u044B\u0439 \u043D\u044E\u0445 \u0438 \u0441\u043B\u0443\u0445. \u0412\u043E \u0432\u0440\u0435\u043C\u044F \u0445\u043E\u0434\u044C\u0431\u044B \u0438 \u0431\u0435\u0433\u0430 \u0435\u0436\u0438 \u043D\u0430\u0441\u0442\u0443\u043F\u0430\u044E\u0442 \u043D\u0430 \u0437\u0435\u043C\u043B\u044E \u0432\u0441\u0435\u0439 \u0441\u0442\u0443\u043F\u043D\u0435\u0439.  \u041B\u0435\u0442\u043E\u043C \u0441\u0435\u0440\u0434\u0446\u0430 \u0435\u0436\u0435\u0439 \u0434\u0435\u043B\u0430\u044E\u0442 180 \u0443\u0434\u0430\u0440\u043E\u0432 \u0432 \u043C\u0438\u043D\u0443\u0442\u0443. \u0417\u0438\u043C\u043E\u0439, \u043A\u043E\u0433\u0434\u0430 \u0436\u0438\u0432\u043E\u0442\u043D\u044B\u0435 \u043D\u0430\u0445\u043E\u0434\u044F\u0442\u0441\u044F \u0432 \u0441\u043F\u044F\u0447\u043A\u0435, \u0441\u0435\u0440\u0434\u0446\u0435\u0431\u0438\u0435\u043D\u0438\u0435 \u0437\u0430\u043C\u0435\u0434\u043B\u044F\u0435\u0442\u0441\u044F \u0434\u043E 20-60 \u0443\u0434\u0430\u0440\u043E\u0432. \u0412\u043E \u0432\u0440\u0435\u043C\u044F \u0441\u043F\u044F\u0447\u043A\u0438 \u0435\u0436\u0438 \u0432\u0434\u044B\u0445\u0430\u044E\u0442 \u0432\u043E\u0437\u0434\u0443\u0445 \u0442\u043E\u043B\u044C\u043A\u043E \u0440\u0430\u0437 \u0432 \u043C\u0438\u043D\u0443\u0442\u0443. \u0422\u0435\u043C\u043F\u0435\u0440\u0430\u0442\u0443\u0440\u0430 \u0438\u0445 \u0442\u0435\u043B \u043D\u0430 \u044D\u0442\u043E\u0442 \u043F\u0435\u0440\u0438\u043E\u0434 \u043F\u0430\u0434\u0430\u0435\u0442 \u0434\u043E 1,8 \u0433\u0440\u0430\u0434\u0443\u0441\u0430. \u0417\u0430 \u043B\u0435\u0442\u043E \u043A\u0430\u0436\u0434\u044B\u0439 \u0435\u0436 \u0434\u043E\u043B\u0436\u0435\u043D \u0437\u0430\u043F\u0430\u0441\u0442\u0438 \u043D\u0435 \u043C\u0435\u043D\u0435\u0435 500 \u0433\u0440\u0430\u043C\u043C\u043E\u0432 \u0436\u0438\u0440\u0430, \u0438\u043D\u0430\u0447\u0435 \u0432\u043E \u0432\u0440\u0435\u043C\u044F \u0441\u043F\u044F\u0447\u043A\u0438 \u0436\u0438\u0432\u043E\u0442\u043D\u044B\u0445 \u0436\u0434\u0435\u0442 \u0433\u043E\u043B\u043E\u0434\u043D\u0430\u044F \u0441\u043C\u0435\u0440\u0442\u044C. \u0415\u0436\u0438 \u2013 \u043E\u0434\u0438\u043D\u043E\u0447\u043D\u044B\u0435 \u0436\u0438\u0432\u043E\u0442\u043D\u044B\u0435, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u0443\u0441\u0442\u0440\u0430\u0438\u0432\u0430\u044E\u0442 \u0433\u043D\u0435\u0437\u0434\u0430 \u043D\u0435\u043F\u043E\u0434\u0430\u043B\u0435\u043A\u0443 \u0434\u0440\u0443\u0433 \u043E\u0442 \u0434\u0440\u0443\u0433\u0430, \u043D\u043E \u0441\u0442\u0430\u0440\u0430\u044E\u0442\u0441\u044F \u043D\u0435 \u043F\u0435\u0440\u0435\u0441\u0435\u043A\u0430\u0442\u044C\u0441\u044F \u0441 \u0434\u0440\u0443\u0433\u0438\u043C\u0438 \u0432\u0437\u0440\u043E\u0441\u043B\u044B\u043C\u0438 \u043E\u0441\u043E\u0431\u044F\u043C\u0438. \u0415\u0436\u0438 \u0432\u0441\u0435\u044F\u0434\u043D\u044B. \u041E\u0441\u043D\u043E\u0432\u0443 \u0438\u0445 \u0440\u0430\u0446\u0438\u043E\u043D\u0430 \u0441\u043E\u0441\u0442\u0430\u0432\u043B\u044F\u044E\u0442 \u0433\u0443\u0441\u0435\u043D\u0438\u0446\u044B, \u0441\u043B\u0438\u0437\u043D\u0438, \u0436\u0443\u043A\u0438, \u0438\u043D\u043E\u0433\u0434\u0430 \u0447\u0435\u0440\u0432\u0438 \u0438 \u043C\u044B\u0448\u0438. \u0427\u0430\u0441\u0442\u043E \u0436\u0435\u0440\u0442\u0432\u0430\u043C\u0438 \u0435\u0436\u0435\u0439 \u0441\u0442\u0430\u043D\u043E\u0432\u044F\u0442\u0441\u044F \u0440\u0435\u043F\u0442\u0438\u043B\u0438\u0438, \u0432\u043F\u0430\u0432\u0448\u0438\u0435 \u0432 \u0441\u043F\u044F\u0447\u043A\u0443. \u0418\u0437 \u0440\u0430\u0441\u0442\u0438\u0442\u0435\u043B\u044C\u043D\u043E\u0439 \u043F\u0438\u0449\u0438 \u0435\u0436\u0438 \u043F\u0440\u0435\u0434\u043F\u043E\u0447\u0438\u0442\u0430\u044E\u0442 \u044F\u0433\u043E\u0434\u044B \u0438 \u0444\u0440\u0443\u043A\u0442\u044B. \u0418\u0441\u0441\u043B\u0435\u0434\u043E\u0432\u0430\u0442\u0435\u043B\u0438 \u0432\u044B\u044F\u0441\u043D\u0438\u043B\u0438, \u0447\u0442\u043E \u0432 \u0441\u043B\u0443\u0447\u0430\u0435 \u043D\u0435\u043E\u0431\u0445\u043E\u0434\u0438\u043C\u043E\u0441\u0442\u0438 \u0451\u0436 \u043C\u043E\u0436\u0435\u0442 \u0441\u044A\u0435\u0441\u0442\u044C \u0433\u0430\u0434\u044E\u043A\u0443. \u0421\u043C\u0435\u0440\u0442\u0435\u043B\u044C\u043D\u044B\u0435 \u0434\u043B\u044F \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0430 \u0434\u043E\u0437\u044B \u043C\u044B\u0448\u044C\u044F\u043A\u0430, \u0441\u0438\u043D\u0438\u043B\u044C\u043D\u043E\u0439 \u043A\u0438\u0441\u043B\u043E\u0442\u044B \u0438 \u0440\u044F\u0434\u0430 \u0434\u0440\u0443\u0433\u0438\u0445 \u044F\u0434\u043E\u0432 \u0431\u0435\u0437\u0432\u0440\u0435\u0434\u043D\u044B \u0434\u043B\u044F \u0435\u0436\u0435\u0439. \u0411\u0435\u0440\u0435\u043C\u0435\u043D\u043D\u043E\u0441\u0442\u044C \u0443 \u0435\u0436\u0435\u0439 \u0434\u043B\u0438\u0442\u0441\u044F 49 \u0434\u043D\u0435\u0439. \u041F\u043E\u0447\u0442\u0438 \u0441\u0440\u0430\u0437\u0443 \u043F\u043E\u0441\u043B\u0435 \u0440\u043E\u0436\u0434\u0435\u043D\u0438\u044F \u043D\u0430 \u0441\u043F\u0438\u043D\u043A\u0430\u0445 \u0435\u0436\u0430\u0442 \u043F\u043E\u044F\u0432\u043B\u044F\u044E\u0442\u0441\u044F \u043C\u044F\u0433\u043A\u0438\u0435 \u043A\u0440\u043E\u0448\u0435\u0447\u043D\u044B\u0435 \u0438\u0433\u043B\u044B. \u0412 \u0432\u043E\u0437\u0440\u0430\u0441\u0442\u0435 \u043E\u0434\u043D\u043E\u0433\u043E \u043C\u0435\u0441\u044F\u0446\u0430 \u0435\u0436\u0430\u0442\u0430 \u0443\u0445\u043E\u0434\u044F\u0442 \u0438\u0437 \u0440\u043E\u0434\u0438\u0442\u0435\u043B\u044C\u0441\u043A\u043E\u0433\u043E \u0433\u043D\u0435\u0437\u0434\u0430 \u0438 \u043D\u0430\u0447\u0438\u043D\u0430\u044E\u0442 \u0436\u0438\u0442\u044C \u0441\u0430\u043C\u043E\u0441\u0442\u043E\u044F\u0442\u0435\u043B\u044C\u043D\u043E. \u0415\u0436\u0438 \u0441\u0442\u0440\u0430\u0434\u0430\u044E\u0442 \u043E\u0442 \u043A\u043B\u0435\u0449\u0435\u0439 \u0431\u043E\u043B\u044C\u0448\u0435, \u0447\u0435\u043C \u043B\u044E\u0431\u044B\u0435 \u0434\u0440\u0443\u0433\u0438\u0435 \u0436\u0438\u0432\u043E\u0442\u043D\u044B\u0435 \u2013 \u0438\u0445 \u043A\u043E\u043B\u044E\u0447\u0430\u044F \u0449\u0435\u0442\u0438\u043D\u0430 \u0438 \u0438\u0433\u043B\u044B \u043F\u0440\u0430\u043A\u0442\u0438\u0447\u0435\u0441\u043A\u0438 \u0432\u044B\u0447\u0435\u0441\u044B\u0432\u0430\u044E\u0442 \u0438\u0437 \u0442\u0440\u0430\u0432\u044B \u0433\u043E\u043B\u043E\u0434\u043D\u044B\u0445 \u043A\u043B\u0435\u0449\u0435\u0439 \u0438 \u0441\u043E\u0431\u0438\u0440\u0430\u044E\u0442 \u043D\u0430 \u0435\u0436\u0438\u043D\u043E\u043C \u0442\u0435\u043B\u0435. \u0412 \u0440\u0435\u0437\u0443\u043B\u044C\u0442\u0430\u0442\u0435 \u043F\u043E\u0441\u0442\u043E\u044F\u043D\u043D\u043E\u0433\u043E \u0432\u0437\u0430\u0438\u043C\u043E\u0434\u0435\u0439\u0441\u0442\u0432\u0438\u044F \u0441 \u044D\u0442\u0438\u043C\u0438 \u043F\u0430\u0440\u0430\u0437\u0438\u0442\u0430\u043C\u0438 \u0435\u0436\u0438 \u0441\u0442\u0430\u043D\u043E\u0432\u044F\u0442\u0441\u044F \u043F\u0435\u0440\u0435\u043D\u043E\u0441\u0447\u0438\u043A\u0430\u043C\u0438 \u043C\u043D\u043E\u0433\u0438\u0445 \u043E\u043F\u0430\u0441\u043D\u044B\u0445 \u0431\u043E\u043B\u0435\u0437\u043D\u0435\u0439. \u0420\u0438\u043C\u043B\u044F\u043D\u0435 \u0435\u0449\u0435 \u0437\u0430 \u043D\u0435\u0441\u043A\u043E\u043B\u044C\u043A\u043E \u0432\u0435\u043A\u043E\u0432 \u0434\u043E \u043D\u0430\u0448\u0435\u0439 \u044D\u0440\u044B \u0440\u0430\u0437\u0432\u043E\u0434\u0438\u043B\u0438 \u0435\u0436\u0435\u0439 \u0440\u0430\u0434\u0438 \u0438\u0445 \u043C\u044F\u0441\u0430 \u2013 \u0437\u0432\u0435\u0440\u044C\u043A\u043E\u0432 \u0432\u043C\u0435\u0441\u0442\u0435 \u0441 \u0438\u0433\u043B\u0430\u043C\u0438 \u0437\u0430\u043F\u0435\u043A\u0430\u043B\u0438 \u0432 \u0433\u043B\u0438\u043D\u0435. \u0415\u0436\u0438 \u043F\u0435\u0440\u0438\u043E\u0434\u0438\u0447\u0435\u0441\u043A\u0438 \u043D\u0430\u043A\u0430\u043B\u044B\u0432\u0430\u044E\u0442 \u043D\u0430 \u0441\u0432\u043E\u0438 \u0438\u0433\u043E\u043B\u043A\u0438 \u043F\u0440\u0435\u0434\u043C\u0435\u0442\u044B \u0441 \u0441\u0438\u043B\u044C\u043D\u044B\u043C \u0437\u0430\u043F\u0430\u0445\u043E\u043C (\u043D\u0430\u043F\u0440\u0438\u043C\u0435\u0440, \u043E\u043A\u0443\u0440\u043A\u0438 \u0441\u0438\u0433\u0430\u0440\u0435\u0442 \u0438\u043B\u0438 \u043A\u0443\u0441\u043E\u0447\u043A\u0438 \u0432\u0430\u0442\u044B \u0441 \u0430\u0440\u043E\u043C\u0430\u0442\u043E\u043C \u0434\u0443\u0445\u043E\u0432). \u0412\u0435\u0440\u043E\u044F\u0442\u043D\u043E, \u0436\u0438\u0432\u043E\u0442\u043D\u044B\u0435 \u0434\u0435\u043B\u0430\u044E\u0442 \u044D\u0442\u043E \u043E\u0442\u043F\u0443\u0433\u0438\u0432\u0430\u043D\u0438\u044F \u043F\u0430\u0440\u0430\u0437\u0438\u0442\u043E\u0432." #@param {type:"string"}
stexts = texts
print(texts)

25 интересных фактов о ежах Интересные факты о ежах  При слове «ёж» многим из нас сразу представляется картинка из детской книжки, на которой весёлое ежиное семейство тащит в норку наколотые на иголки грибочки. На самом же деле ежи — это хитрые и умные хищники, которые охотно питаются не только грибами и ягодами, но и вообще всем, что сумеют найти. Или кого сумеют поймать.  Интересные факты о ежах На верхней челюсти ежей расположены 20 маленьких зубов, на нижней – 16. При длине тела в 20-30 сантиметров у ежей довольно маленькие уши – меньше 3,5 сантиметров. Уши большего размера встречаются только у ежей, живущих на Кипре (см. интересные факты о Кипре). Внутри иглы ежей полые – они наполнены только воздухом. Тела молодых ежей защищают от хищников около 3 000 иголок, взрослых – 5-6 тысяч иголок. В последние годы все больше ежей перебирается из лесов в города. Ученые связывают это с тем, что на городских улицах не водятся основные враги ежей – барсуки и совы (см. интересные факты о совах)

In [2]:
# !pip install scikit-learn
!pip install pandas==1.2.3
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

     |████████████████████████████████| 9.9 MB 10.3 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [3]:
def printTF_IDF(texts):
  records_count = 30
  tfIdfTransformer = TfidfVectorizer(ngram_range=(1, 4), use_idf=True, max_features=records_count)
  countVectorizer = CountVectorizer(ngram_range=(1, 4), max_features=records_count)
  wordCount = countVectorizer.fit_transform([texts])
  TfIdf = tfIdfTransformer.fit_transform([texts])
  names = countVectorizer.get_feature_names()
  df=[]
  df = pd.DataFrame(list(names), columns=['names'])
  df = df.assign(Word_Count=wordCount.T.todense())
  df = df.assign(TF_IDF=TfIdf.T.todense())
  df = df.sort_values('TF_IDF', ascending=False)
  print(df)

In [4]:
printTF_IDF(texts)

               names  Word_Count    TF_IDF
7                ежи          17  0.503939
18                на          15  0.444652
6               ежей          13  0.385365
4                 до           8  0.237148
20                от           8  0.237148
11                из           6  0.177861
8           животные           5  0.148217
19                не           5  0.148217
10              иглы           5  0.148217
0                 20           4  0.118574
14  интересные факты           4  0.118574
26             факты           4  0.118574
24            только           4  0.118574
15                их           4  0.118574
13        интересные           4  0.118574
1                 во           3  0.088930
16           которые           3  0.088930
17               лет           3  0.088930
12               или           3  0.088930
9                 за           3  0.088930
5               ежах           3  0.088930
21       сантиметров           3  0.088930
3          

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


удалим знаки переноса строк и сделаем текст в нижнем регистре

In [5]:
  str2 = ''
  for item in texts.split():
    str2 = str2 + ' ' + item
  texts = str(texts)
  texts = texts.lower()
  # print(texts)
  texts = texts.replace('\n', ' ')
  print(texts)

25 интересных фактов о ежах интересные факты о ежах  при слове «ёж» многим из нас сразу представляется картинка из детской книжки, на которой весёлое ежиное семейство тащит в норку наколотые на иголки грибочки. на самом же деле ежи — это хитрые и умные хищники, которые охотно питаются не только грибами и ягодами, но и вообще всем, что сумеют найти. или кого сумеют поймать.  интересные факты о ежах на верхней челюсти ежей расположены 20 маленьких зубов, на нижней – 16. при длине тела в 20-30 сантиметров у ежей довольно маленькие уши – меньше 3,5 сантиметров. уши большего размера встречаются только у ежей, живущих на кипре (см. интересные факты о кипре). внутри иглы ежей полые – они наполнены только воздухом. тела молодых ежей защищают от хищников около 3 000 иголок, взрослых – 5-6 тысяч иголок. в последние годы все больше ежей перебирается из лесов в города. ученые связывают это с тем, что на городских улицах не водятся основные враги ежей – барсуки и совы (см. интересные факты о совах)

удалим цифры

In [6]:
  str2 = ''
  for c in texts:
      if c not in ('0', "1", '2', '3', '4', '5', '6', '7', '8', '9', '«', '»', '–', "\""):
          str2 = str2 + c
  texts = str2
  str2 = ''
  print(texts)

 интересных фактов о ежах интересные факты о ежах  при слове ёж многим из нас сразу представляется картинка из детской книжки, на которой весёлое ежиное семейство тащит в норку наколотые на иголки грибочки. на самом же деле ежи — это хитрые и умные хищники, которые охотно питаются не только грибами и ягодами, но и вообще всем, что сумеют найти. или кого сумеют поймать.  интересные факты о ежах на верхней челюсти ежей расположены  маленьких зубов, на нижней  . при длине тела в - сантиметров у ежей довольно маленькие уши  меньше , сантиметров. уши большего размера встречаются только у ежей, живущих на кипре (см. интересные факты о кипре). внутри иглы ежей полые  они наполнены только воздухом. тела молодых ежей защищают от хищников около   иголок, взрослых  - тысяч иголок. в последние годы все больше ежей перебирается из лесов в города. ученые связывают это с тем, что на городских улицах не водятся основные враги ежей  барсуки и совы (см. интересные факты о совах). ежи  ночные животные, к

удалим знаки пунктуации

In [7]:
import string
pattern = string.punctuation
for c in texts:
    if c not in pattern:
        str2 = str2 + c
    else:
        str2 = str2 + " "
texts = str2
str2 = ''
print(texts)

 интересных фактов о ежах интересные факты о ежах  при слове ёж многим из нас сразу представляется картинка из детской книжки  на которой весёлое ежиное семейство тащит в норку наколотые на иголки грибочки  на самом же деле ежи — это хитрые и умные хищники  которые охотно питаются не только грибами и ягодами  но и вообще всем  что сумеют найти  или кого сумеют поймать   интересные факты о ежах на верхней челюсти ежей расположены  маленьких зубов  на нижней    при длине тела в   сантиметров у ежей довольно маленькие уши  меньше   сантиметров  уши большего размера встречаются только у ежей  живущих на кипре  см  интересные факты о кипре   внутри иглы ежей полые  они наполнены только воздухом  тела молодых ежей защищают от хищников около   иголок  взрослых    тысяч иголок  в последние годы все больше ежей перебирается из лесов в города  ученые связывают это с тем  что на городских улицах не водятся основные враги ежей  барсуки и совы  см  интересные факты о совах   ежи  ночные животные  к

загрузим стоп слова, используем библиотеку NLTK

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Добавить свои стоп-слова:

In [9]:
my_stop_words = "\u0420\u0443\u0441\u0430\u043A\u043E\u0432 \u041C\u0438\u0445\u0430\u043B\u044B\u0447" #@param {type:"string"}
print(my_stop_words)

Русаков Михалыч


Удалим стоп слова из текста

In [10]:
  from nltk.corpus import stopwords
  russian_stopwords = stopwords.words("russian")

  for word in my_stop_words.split():
    russian_stopwords.append(word)
  str2 = ''
  for word in texts.split():
      if word not in (russian_stopwords):
          str2 = str2 + " " + word
  texts = str2
  str2 = ''
  for word in texts.split():
      if len(word) > 1:
          str2 = str2 + " " + word
  texts = str2
  print(russian_stopwords)

  

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

рассчитаем TF-IDF для отфильтрованных данных

In [11]:
printTF_IDF(texts)

                  names  Word_Count    TF_IDF
7                   ежи          17  0.650487
6                  ежей          13  0.497431
8              животные           5  0.191320
10                 иглы           5  0.191320
12           интересные           4  0.153056
26                факты           4  0.153056
13     интересные факты           4  0.153056
0                 время           3  0.114792
28                  это           3  0.114792
18          сантиметров           3  0.114792
17                  лет           3  0.114792
16              которые           3  0.114792
29                   ёж           3  0.114792
5                  ежах           3  0.114792
11               иголок           2  0.076528
22               спячки           2  0.076528
2                    га           2  0.076528
27           факты ежах           2  0.076528
3                гнезда           2  0.076528
25               ударов           2  0.076528
24           становятся           

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


#Лемматизация
Лемматизация (англ. lemmatization) - это метод морфологического анализа, который сводится к приведению словоформы к ее первоначальной словарной форме (лемме).

Метод лемматизации применяется в поисковых алгоритмах в процессе схематизации веб-документов при их индексировании.

В результате лемматизации от словоформы отбрасываются флективные окончания и возвращается основная или словарная форма слова.

Например, в русском языке словарной формой считается:

существительные - именительный падеж, единственное число (мечами - меч)
глаголы - инфинитивная форма (читали - читать)
прилагательные - единственное число, именительный падеж, мужской род (заснеженными - заснеженный)

In [12]:
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
ltexts = ""
print(texts)
for word in texts.split():
    if len(str(word)) > 2:
        ltexts+=" " + morph.parse(word)[0].normal_form
print(ltexts)

     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 8.2 MB 12.4 MB/s 
 интересных фактов ежах интересные факты ежах слове ёж многим сразу представляется картинка детской книжки которой весёлое ежиное семейство тащит норку наколотые иголки грибочки самом деле ежи это хитрые умные хищники которые охотно питаются грибами ягодами вообще всем сумеют найти кого сумеют поймать интересные факты ежах верхней челюсти ежей расположены маленьких зубов нижней длине тела сантиметров ежей довольно маленькие уши меньше сантиметров уши большего размера встречаются ежей живущих кипре см интересные факты кипре внутри иглы ежей полые наполнены воздухом тела молодых ежей защищают хищников около иголок взрослых тысяч иголок последние годы ежей перебирается лесов города ученые связывают это городских улицах водятся основные враги ежей барсуки совы см интересные факты совах ежи ночные животные которые любят отходить далеко своего жилища день ежи обычно проводят гнез

рассчитаем TF-IDF для Лемматизированных данных

In [13]:
printTF_IDF(ltexts)

                 names  Word_Count    TF_IDF
7                  ежи          17  0.553596
29                  ёж          16  0.521032
10                игла           7  0.227951
4                  год           6  0.195387
8             животное           6  0.195387
12          интересный           5  0.162822
25                факт           5  0.162822
24                тело           5  0.162822
13     интересный факт           5  0.162822
22              спячка           4  0.130258
20                свой           4  0.130258
15             который           4  0.130258
11              иголка           4  0.130258
3               гнездо           3  0.097693
14  интересный факт ёж           3  0.097693
27                 это           3  0.097693
17           маленький           3  0.097693
26             факт ёж           3  0.097693
19           сантиметр           3  0.097693
6               другой           3  0.097693
9                 жить           3  0.097693
2         

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


#Суммаризации текстов 

Задача суммаризации текстов (автореферирование) - одна из ключевых, широко обсуждаемых задач NLP. Она состоит в сжатии больших объемов текста до связного краткого содержания, отражающего только основные идеи.


#Алгоритм TextRank
TextRank - это алгоритм, основанный на PageRank, который часто используется для извлечения ключевых слов и суммирования текста. 
PageRank (PR) - это алгоритм, используемый для расчета веса веб-страниц. 


In [14]:
from gensim.summarization import summarize, keywords
print("Исходный текст:")
print(stexts)
print("\nРезультат работы TextRank:")
print(summarize(str(stexts)))

Исходный текст:
25 интересных фактов о ежах Интересные факты о ежах  При слове «ёж» многим из нас сразу представляется картинка из детской книжки, на которой весёлое ежиное семейство тащит в норку наколотые на иголки грибочки. На самом же деле ежи — это хитрые и умные хищники, которые охотно питаются не только грибами и ягодами, но и вообще всем, что сумеют найти. Или кого сумеют поймать.  Интересные факты о ежах На верхней челюсти ежей расположены 20 маленьких зубов, на нижней – 16. При длине тела в 20-30 сантиметров у ежей довольно маленькие уши – меньше 3,5 сантиметров. Уши большего размера встречаются только у ежей, живущих на Кипре (см. интересные факты о Кипре). Внутри иглы ежей полые – они наполнены только воздухом. Тела молодых ежей защищают от хищников около 3 000 иголок, взрослых – 5-6 тысяч иголок. В последние годы все больше ежей перебирается из лесов в города. Ученые связывают это с тем, что на городских улицах не водятся основные враги ежей – барсуки и совы (см. интересны

#Алгоритм Rake
RAKE: Rapid Automatic Keyword Extraction Algorithm
Алгоритм RAKE извлекает ключевые слова с помощью основанного на разделителе подхода, чтобы идентифицировать ключевые слова кандидата и баллы их использующий совместные встречаемости слова, которые появляются в ключевых словах кандидата. Ключевые слова могут содержать несколько лексем. Кроме того, алгоритм RAKE также объединяет ключевые слова, когда они кажутся многократно, разделенными тем же разделителем слияния.


In [15]:
!pip install rake_nltk
from rake_nltk import Metric, Rake
r = Rake(language="russian")
import nltk
nltk.download('punkt')
r.extract_keywords_from_text(stexts)
mas = r.get_ranked_phrases()
set2 = set()
for item in mas:
    if not "nan" in str(item).replace(" nan ", " "):
        set2.add(str(item).replace(" nan ", " "))
mas = list(set2)
print("Исходный текст:")
print(stexts)
print("\nРезультат работы TextRammk:")
for item in mas:
  print(item)
# print(str(mas))

     |████████████████████████████████| 1.5 MB 8.3 MB/s 
     |████████████████████████████████| 749 kB 38.3 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Исходный текст:
25 интересных фактов о ежах Интересные факты о ежах  При слове «ёж» многим из нас сразу представляется картинка из детской книжки, на которой весёлое ежиное семейство тащит в норку наколотые на иголки грибочки. На самом же деле ежи — это хитрые и умные хищники, которые охотно питаются не только грибами и ягодами, но и вообще всем, что сумеют найти. Или кого сумеют поймать.  Интересные факты о ежах На верхней челюсти ежей расположены 20 маленьких зубов, на нижней – 16. При длине тела в 20-30 сантиметров у ежей довольно маленькие уши – меньше 3,5 сантиметров. Уши большего размера встречаются только у ежей, живущих на Кипре (см. интересные факты о Кипре). Внутри иглы ежей полые – они наполнены только воздухом. Тела молодых ежей защищают от хищников около 3 000 иголок, взрослых – 5-6 тысяч иголок. В последние годы все больше ежей перебирается из лесов в города. Ученые с

#Тематическое моделирование
Тематическое моделирование — это метод извлечения тем из текста. Latent Dirichlet Allocation (LDA) — популярный алгоритм моделирования тем реализованные в том числе в пакете Gensim. Основная задача алгоритмов ТМ, заключается в том что бы полученные темы были хорошего качество, понятными, самозначимыми и разделенными. Достижение этих целей во многом зависит от качества предварительной обработки текста и стратегии поиска оптимального количества тем. 

##Импорт пакетов


In [16]:
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Подход LDA к тематическому моделированию заключается в том, что каждый документ рассматривается как набор тем в определенной пропорции. И каждая тема как набор ключевых слов, опять же, в определенной пропорции.

После того, как вы укажете алгоритму количество тем, все, что он сделает, — это отобразит распределение тем в документах и распределение ключевых слов по темам.

Тема — это не что иное, как набор доминирующих ключевых слов. Просто взглянув на ключевые слова, вы сможете определить, о чем эта тема.

Ниже приведены ключевые факторы для получения хороших разделительных тем:

Качество обработки текста.
Разнообразие тем, о которых говорится в тексте.
Выбор алгоритма моделирование тем.
Количество тем, указанных в алгоритме.
Алгоритмы настройки параметров.

# Подготовим стоп-слова


In [17]:
from nltk.corpus import stopwords
stop_words = russian_stopwords

##Лемматизация
Лемматизация — это не что иное, как преобразование слова в его корневое слово. Например: лемма слова «machines» — это «machine». Аналогично, «walking» -> «walk», «mice» -> «mouse» и так далее.

In [18]:
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
ltexts = ""
print(texts)
for word in texts.split():
    if len(str(word)) > 2:
        ltexts+=" " + morph.parse(word)[0].normal_form
print(ltexts)

 интересных фактов ежах интересные факты ежах слове ёж многим сразу представляется картинка детской книжки которой весёлое ежиное семейство тащит норку наколотые иголки грибочки самом деле ежи это хитрые умные хищники которые охотно питаются грибами ягодами вообще всем сумеют найти кого сумеют поймать интересные факты ежах верхней челюсти ежей расположены маленьких зубов нижней длине тела сантиметров ежей довольно маленькие уши меньше сантиметров уши большего размера встречаются ежей живущих кипре см интересные факты кипре внутри иглы ежей полые наполнены воздухом тела молодых ежей защищают хищников около иголок взрослых тысяч иголок последние годы ежей перебирается лесов города ученые связывают это городских улицах водятся основные враги ежей барсуки совы см интересные факты совах ежи ночные животные которые любят отходить далеко своего жилища день ежи обычно проводят гнезде имеющем диаметре сантиметров ежи очищают расправляют свои иглы длинными пальцами ногах ежи ревностно оберегают 

##Создадим словарь и корпус.
Двумя основными входными данными для тематической модели LDA являются словарь (id2word) и корпус. Давайте создадим их.

In [19]:
from nltk.util import ngrams
def make_bigrams(ttexts):
    texts = ' '.join(ttexts)
    token = nltk.word_tokenize(ttexts)
    bigrams = list(ngrams(token, 2))
    # print(bigrams)
    return bigrams
def make_trigrams(ttexts):
    texts = ' '.join(ttexts)
    token = nltk.word_tokenize(ttexts)
    trigrams = list(ngrams(token, 2))
    return trigrams

btexts = make_bigrams(texts) + make_trigrams(texts)
id2word = corpora.Dictionary(btexts)
corpus = [id2word.doc2bow(text) for text in btexts]
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('интересных', 1), ('фактов', 1)]]

Или вы можете увидеть удобочитаемую форму самого корпуса.

In [20]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('интересных', 1), ('фактов', 1)]]

##Построим тематическую модель
У нас есть все необходимое для обучения модели LDA. В дополнение к корпусу и словарю необходимо также указать количество тем.

In [21]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3,                          # Количество тем (отдельная тема для диссертации)
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

##Просмотр тем в модели LDA
Вышеупомянутая модель LDA построена на 20 различных темах, где каждая тема представляет собой комбинацию ключевых слов, и каждое ключевое слово вносит определенный вес в тему.

Вы можете увидеть ключевые слова для каждой темы и вес (важность) каждого ключевого слова, используя lda_model.print_topics().

In [22]:
lda_model.print_topics()

[(0,
  '0.095*"ежи" + 0.034*"животные" + 0.020*"клещей" + 0.014*"иголки" + 0.013*"факты" + 0.013*"интересные" + 0.010*"несколько" + 0.010*"опасных" + 0.010*"болезней" + 0.010*"римляне"'),
 (1,
  '0.037*"иглы" + 0.021*"гнезда" + 0.021*"становятся" + 0.020*"делают" + 0.015*"сантиметров" + 0.014*"минуту" + 0.014*"ударов" + 0.010*"см" + 0.009*"тела" + 0.009*"сумеют"'),
 (2,
  '0.082*"ежей" + 0.023*"это" + 0.022*"время" + 0.020*"теле" + 0.020*"свои" + 0.018*"лет" + 0.018*"ёж" + 0.018*"которые" + 0.013*"сразу" + 0.012*"условиях"')]

##Визуализация темы и ключевых слов
Теперь, когда модель LDA создана, следующим шагом является изучение созданных тем и связанных с ними ключевых слов. Нет лучшего инструмента, чем интерактивная диаграмма пакета pyLDAvis.

In [23]:
!pip install pyLDAvis==2.1.2
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

     |████████████████████████████████| 1.6 MB 4.6 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97738 sha256=191401460e0dffad2a9cd44e3f3b3f045bf6323d7f596fb5701624d54549ebb5
  Stored in directory: /root/.cache/pip/wheels/3b/fb/41/e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
Successfully built pyLDAvis


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [24]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.107444  0.203864       1        1  35.131391
0     -0.240896 -0.014163       2        1  33.554926
1      0.133452 -0.189701       3        1  31.313683, topic_info=            Term       Freq      Total Category  logprob  loglift
24           ежи  49.000000  49.000000  Default  30.0000  30.0000
42          ежей  44.000000  44.000000  Default  29.0000  29.0000
61          иглы  18.000000  18.000000  Default  28.0000  28.0000
88      животные  18.000000  18.000000  Default  27.0000  27.0000
25           это  12.000000  12.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
178  температура   3.879244   4.442288   Topic3  -4.8228   1.0256
180       период   3.879244   4.442288   Topic3  -4.8228   1.0256
181       падает   3.879244   4.442288   Topic3  -4.8228   1.0256
232       гадюку   3.879244   4.442288   Topic3  -4.8228   1.0256
229       случае   3.879244   4.442288   Topic3  -4.8228   1.0256

[181 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
243       1  0.936384    беременность
281       2  0.837366        болезней
284       2  0.837366           веков
275       2  0.837366  взаимодействия
205       3  0.900437       взрослыми
...     ...       ...             ...
25        1  0.930629             это
146       1  0.746650        являются
225       3  0.900437           ягоды
241       3  0.900437            ядов
6         1  0.983763              ёж

[161 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])